In [153]:
import numpy as np
import pandas as pd

In [154]:
# Define the KNN class with distance-weighted voting
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def compute_distance(self, X1, X2):
        if self.distance_metric == 'euclidean':
            return np.linalg.norm(X2 - X1, axis=1)
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X2 - X1), axis=1)
        else:
            raise ValueError("Unsupported distance metric")

    def predict(self, X):
        return np.apply_along_axis(self.pred, 1, X.values)

    def pred(self, x):
        distances = self.compute_distance(x, self.X_train.values)
        k_indices = np.argpartition(distances, self.k)[:self.k]
        k_nearest_labels = self.y_train.iloc[k_indices]
        
        # Distance-weighted averaging
        nearest_distances = distances[k_indices]
        weights = 1 / (nearest_distances + 1e-5)  # Avoid division by zero
        weighted_average = np.dot(weights, k_nearest_labels) / np.sum(weights)
        
        return weighted_average  # Return probability

In [155]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    combined_data = pd.concat([train_data, test_data], axis=0, ignore_index=True)

    # Encode categorical variables
    combined_data = pd.get_dummies(combined_data, columns=['Geography', 'Gender'], drop_first=True)

    # Define feature columns
    features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 
                'HasCrCard', 'IsActiveMember', 'EstimatedSalary'] + \
               [col for col in combined_data.columns if col.startswith('Geography_') or col.startswith('Gender_')]

    # Normalize features
    combined_data[features] = (combined_data[features] - combined_data[features].mean()) / combined_data[features].std()

    X_train = combined_data.loc[:len(train_data)-1, features]
    y_train = train_data['Exited']
    X_test = combined_data.loc[len(train_data):, features]

    return X_train, y_train, X_test

In [156]:
# Define custom AUC scoring function
def auc_score(y_true, y_est_prob):
    positive_indices = np.where(y_true == 1)[0]
    negative_indices = np.where(y_true == 0)[0]

    if len(positive_indices) == 0 or len(negative_indices) == 0:
        return 0.5

    positive_predictions = y_est_prob[positive_indices]
    negative_predictions = y_est_prob[negative_indices]

    correct_order = np.sum(positive_predictions[:, None] > negative_predictions)
    return correct_order / (len(positive_indices) * len(negative_indices))

# Perform cross-validation with weighted voting
def cross_validate(X, y, knn, n_splits=5):
    np.random.seed(42)
    indices = np.random.permutation(len(X))
    fold_size = len(X) // n_splits
    scores = []

    for i in range(n_splits):
        val_indices = indices[i * fold_size:(i + 1) * fold_size]
        train_indices = np.setdiff1d(indices, val_indices)

        X_train, X_val = X.iloc[train_indices], X.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]

        knn.fit(X_train, y_train)
        y_est_prob = knn.predict(X_val)
        score = auc_score(y_val, y_est_prob)
        scores.append(score)

    return scores

In [157]:
# Load and preprocess data
X, y, X_test = preprocess_data('/Users/nathanlee/Desktop/CS506/train.csv', '/Users/nathanlee/Desktop/CS506/test.csv')

# Hyperparameter tuning with more k values
k_values = [3, 5, 7, 9, 11, 13, 15, 17, 19]
distance_metrics = ['euclidean', 'manhattan']
best_score = 0
best_params = {}

for k in k_values:
    for metric in distance_metrics:
        knn = KNN(k=k, distance_metric=metric)
        scores = cross_validate(X, y, knn)
        mean_score = np.mean(scores)

        if mean_score > best_score:
            best_score = mean_score
            best_params = {'k': k, 'distance_metric': metric}

print("Best parameters:", best_params)
print("Best ROC AUC score:", best_score)

# Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(**best_params)
knn.fit(X, y)
test_predictions = knn.predict(X_test)

# Save the probability predictions to submissions.csv
submission_df = pd.DataFrame({'id': pd.read_csv('/Users/nathanlee/Desktop/CS506/test.csv')['id'], 'Exited': test_predictions})
submission_df.to_csv('/Users/nathanlee/Desktop/CS506/submissions.csv', index=False)

Best parameters: {'k': 19, 'distance_metric': 'euclidean'}
Best ROC AUC score: 0.6958672176585978
